In [1]:
!pip install starfish

     |████████████████████████████████| 245kB 5.3MB/s 
     |████████████████████████████████| 235kB 7.0MB/s 
     |████████████████████████████████| 26.3MB 178kB/s 
     |████████████████████████████████| 133kB 46.4MB/s 
     |████████████████████████████████| 133kB 46.1MB/s 
     |████████████████████████████████| 51kB 5.9MB/s 
     |████████████████████████████████| 6.8MB 36.6MB/s 
     |████████████████████████████████| 71kB 7.7MB/s 
  Created wheel for starfish: filename=starfish-0.2.1-cp36-none-any.whl size=367892 sha256=4e0ac873e3b701d601c3578243c499b5794a8a8815e9315a8c51dedaf871a531
  Stored in directory: /root/.cache/pip/wheels/65/78/7c/f166fe3bd1c40e85b2daaf7f71346b76c649ff023ee0d5a8cd
  Created wheel for read-roi: filename=read_roi-1.6.0-py2.py3-none-any.whl size=8031 sha256=38451702f05f204b496b6078aef8fa539f421e803f9dcb992ffeaf561b93fe9c
  Stored in directory: /root/.cache/pip/wheels/9b/65/9c/11e19146187721d543eb606499e0a7abe52e48ecb73efaedbb
  Created wheel for regional: f

# Load image and convert to 5D

In [1]:

from typing import Mapping, Union



from starfish.core.types import Coordinates, CoordinateValue, Axes
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
import dask
import dask.array as da
from skimage import (
     io
)
from google.colab import files

uploaded = files.upload()#creating file input stream

for fn in uploaded.keys():#getting the input bytes
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))#get name and length of file
data = io.imread('3mem_3m0_1-1 ROI 1_crop_crop_crop_Cy3.tif')#import file path
z,y,x=data.shape#getting image shape
print(z,y,x)
print(data.ndim)
print(data.dtype)

z,y,x = data.shape
num_r = 1
num_c=1
synthetic_data=np.resize(data, (num_r,num_c,z,y,x)) 
print(synthetic_data.shape)
print(synthetic_data.ndim)


Saving 3mem_3m0_1-1 ROI 1_crop_crop_crop_Cy3.tif to 3mem_3m0_1-1 ROI 1_crop_crop_crop_Cy3.tif
User uploaded file "3mem_3m0_1-1 ROI 1_crop_crop_crop_Cy3.tif" with length 18726912 bytes
94 314 315
3
uint16
(1, 1, 94, 314, 315)
5


# Save the 5D file and load in space tx format

In [59]:
from starfish import ImageStack
stack=ImageStack.from_numpy(synthetic_data)#converting numpy array to starFish format
print(stack)


<starfish.ImageStack (r: 1, c: 1, z: 94, y: 314, x: 315)>


/usr/local/lib/python3.6/dist-packages/starfish/core/imagestack/imagestack.py:413: UserWarning: ImageStack detected as uint16. Converting to float32...
  warnings.warn(f"ImageStack detected as {array.dtype}. Converting to float32...")


# Spot finding algo 

In [63]:
from starfish import data
from starfish import FieldOfView
from starfish.image import Filter
from starfish.spots import FindSpots

img=stack

# filter to remove noise, remove background, blur, and clip
bandpass = Filter.Bandpass(lshort=.5, llong=7, threshold=0.0)
glp = Filter.GaussianLowPass(
    sigma=(1, 0, 0),
    is_volume=True
)
clip1 = Filter.Clip(p_min=50, p_max=100)
clip2 = Filter.Clip(p_min=99, p_max=100, is_volume=True)
clip1.run(img, in_place=True)
bandpass.run(img, in_place=True)
glp.run(img, in_place=True)
clip2.run(img, in_place=True)


tlmpf = FindSpots.TrackpyLocalMaxPeakFinder(
    spot_diameter=5,  # must be odd integer
    min_mass=0.02,
    max_size=2,  # this is max radius
    separation=7,
    preprocess=False,
    percentile=10,  # this has no effect when min_mass, spot_diameter, and max_size are set properly
    verbose=True,
)
spots = tlmpf.run(img)

# Display feature matrix (spots found)

In [16]:
from starfish.core.spots.DecodeSpots.trace_builders import build_spot_traces_exact_match
intensity_table = build_spot_traces_exact_match(spots)#build intensity table
print(intensity_table)
print(intensity_table.shape)

<xarray.IntensityTable (features: 237, r: 1, c: 1)>
array([[[0.02912611]],

       [[0.04213389]],

       [[0.02460167]],

       [[0.01640111]],

       [[0.02969167]],

       [[0.03845778]],

       [[0.02601556]],

       [[0.02233945]],

       [[0.00650389]],

       [[0.04920334]],

       [[0.02233945]],

       [[0.03647833]],

       [[0.03365056]],

       [[0.02912611]],

       [[0.05796945]],

       [[0.04043722]],

       [[0.04439611]],

       [[0.01498722]],

       [[0.04750667]],

       [[0.02884333]],

       [[0.03619556]],

       [[0.04581   ]],

       [[0.04100278]],

       [[0.02545   ]],

       [[0.03930611]],

       [[0.06221111]],

       [[0.01385611]],

       [[0.03958889]],

       [[0.05146556]],

       [[0.00282778]],

       [[0.02516722]],

       [[0.03336778]],

       [[0.00876611]],

       [[0.04722389]],

       [[0.01979445]],

       [[0.01894611]],

       [[0.01498722]],

       [[0.02460167]],

       [[0.02177389]],

       [[0.0

# Creating synthetic codebook (originally has to be provided by user)

In [17]:
 from starfish import Codebook
 sd = Codebook.synthetic_one_hot_codebook(n_round=1, n_channel=1, n_codes=1)#create code book of same number of rounds and channels present
 print(sd)

<xarray.Codebook (target: 1, r: 1, c: 1)>
array([[[1]]], dtype=uint8)
Coordinates:
  * target   (target) object 13eed58b-59b8-4ef9-ba6d-8f43de4b047a
  * r        (r) int64 0
  * c        (c) int64 0


# Decoding by Lookup Decoder

In [18]:
from starfish.spots import DecodeSpots
decoder = DecodeSpots.SimpleLookupDecoder(codebook=sd)#decode the spots based on codebook codewords
decoded_intensities = decoder.run(spots=spots)

# Printing the decoded intensities 
Here it is one image round with one channel hence the spots detected are  mapped to only 1 code from the codebook.

In [19]:
print(decoded_intensities)

<xarray.DecodedIntensityTable (features: 237, r: 1, c: 1)>
array([[[0.02912611]],

       [[0.04213389]],

       [[0.02460167]],

       [[0.01640111]],

       [[0.02969167]],

       [[0.03845778]],

       [[0.02601556]],

       [[0.02233945]],

       [[0.00650389]],

       [[0.04920334]],

       [[0.02233945]],

       [[0.03647833]],

       [[0.03365056]],

       [[0.02912611]],

       [[0.05796945]],

       [[0.04043722]],

       [[0.04439611]],

       [[0.01498722]],

       [[0.04750667]],

       [[0.02884333]],

       [[0.03619556]],

       [[0.04581   ]],

       [[0.04100278]],

       [[0.02545   ]],

       [[0.03930611]],

       [[0.06221111]],

       [[0.01385611]],

       [[0.03958889]],

       [[0.05146556]],

       [[0.00282778]],

       [[0.02516722]],

       [[0.03336778]],

       [[0.00876611]],

       [[0.04722389]],

       [[0.01979445]],

       [[0.01894611]],

       [[0.01498722]],

       [[0.02460167]],

       [[0.02177389]],

     

# Save coordinates -> target(based on codebook) as csv file

In [12]:
decoded_intensities.to_mermaid('intensity_table.csv.gz')

In [13]:
files.download('intensity_table.csv.gz')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>